Regressione Multipla Lineare
============
L'obiettivo è quello di stimare il valore potenziale sulla base di una serie di variabili quantitative mediante modello di regressione multipla llineare. 
Il valore potenziale è una variabile discreta. Supponiamo volessimo imputare i valori mancanti nel dataset  di una ariabile quantitativa utilizzeremo un'algoritmo di Machine learning per stimare il valore mancante.

Utilizzeremo il dataset di [kaggle](https://www.kaggle.com/mathan/fifa-2018-match-statistics)  relativo alle partite disputate durante i mondiali di calcio nel 2018. 

Il dataset è presente sul sito di kaggle relativo aL problema di individuare il man of the match:
>*(https://www.kaggle.com/mathan/fifa-2018-match-statistics)*

Perchè questo modello ?
E’ la generalizzazione del modello di regressione lineare semplice: per spiegare il fenomeno d’interesse $Y$ spiegato da un numero $p$ di variabili esplicative, con $p > 1$ si usa il modello di Regressione multipla. Tale generalizzazione diventa molto più semplice utilizzando l’algebra delle matrici. Tuttavia il modello di regressione
multipla presenta nuovi problemi:

- Quali variabili scegliere
- Valutare la collinearità delle variabili 
- Test multipli


<img src= "damiele logo.jpg"/>


L'equazione del modello di regressione multipla che andremo a implementare è la seguente:

$y=β_0 +β_1X_1+....+ β_pX_p + Ɛ $

Dove $Ɛ$ è la parte relativa all'errore del modello (differenza tra valori stimati e osservati).

Per ogni elementi $i-esimo$ avremo:

$y_i = βˆ_0 + βˆ_1x_i1 + βˆ_
2x_i2 + . . . + βˆ_
px_ip + ˆe_i = ˆy_i + ˆe_i$


Questa equazione definisce un iperpiano nello spazio a $p + 1$ dimensioni simile alla figura qui sotto.



<img src="Regressione.jpg"/>


Per avere
un’idea del procedimento di stima dei minimi quadrati, il piano
rappresentato in figura è, tra gli infiniti piani, quello che rende
minima la somma dei quadrati delle lunghezze dei segmenti
congiungenti i punti osservati al piano stesso. 

Per stimare i parametri del modello di regressione multipla, senza
fare ulteriori assunzioni circa la forma distributiva degli errori, si
utilizza il metodo dei minimi quadrati (LS).

In [122]:
#Importo le librerie che mi serviranno
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [123]:
#Carico i dati e visualizzo le prime righe per vedere di cosa è composto il mio dataframe
fifa=pd.read_csv("fifa_data.csv")
fifa.head()

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,2202,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,20801,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,€405K,2228,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,190871,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,2143,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,193080,De Gea,27,Spain,91,93,Manchester United,€72M,€260K,1471,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,192985,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,€355K,2281,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


Vediamo quali sono le variabili in generale e di che tipo per farci un'idea anche di quali abbiano più dati mancanti.
Dovrà tenere le variabili quantitative e se ci sono variabili qualitative che possono essere convertite le tengo altrimenti le rimuovo.

In [124]:
#LA variabile che sceglieremo come target è Potential
fifa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18207 entries, 0 to 18206
Data columns (total 85 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        18207 non-null  int64  
 1   Name                      18207 non-null  object 
 2   Age                       18207 non-null  int64  
 3   Nationality               18207 non-null  object 
 4   Overall                   18207 non-null  int64  
 5   Potential                 18207 non-null  int64  
 6   Club                      17966 non-null  object 
 7   Value                     18207 non-null  object 
 8   Wage                      18207 non-null  object 
 9   Special                   18207 non-null  int64  
 10  Preferred Foot            18159 non-null  object 
 11  International Reputation  18159 non-null  float64
 12  Weak Foot                 18159 non-null  float64
 13  Skill Moves               18159 non-null  float64
 14  Work R

Poichè i portieri sono troppo differenti dagli altri (hanno delle caratteristiche troppo diverse) farò un'analisi solo sui giocatori di movimenti perchè alcune variabili del nostro dataset sono troppo peculiari di un ruolo specifico e le andrò ad escludere

In [125]:
fifa.Position.unique()

array(['RF', 'ST', 'LW', 'GK', 'RCM', 'LF', 'RS', 'RCB', 'LCM', 'CB',
       'LDM', 'CAM', 'CDM', 'LS', 'LCB', 'RM', 'LAM', 'LM', 'LB', 'RDM',
       'RW', 'CM', 'RB', 'RAM', 'CF', 'RWB', 'LWB', nan], dtype=object)

In [126]:
fifa=fifa[fifa["Position"]!="GK"]
fifa

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,2202,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,20801,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,€405K,2228,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,190871,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,2143,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
4,192985,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,€355K,2281,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
5,183277,E. Hazard,27,Belgium,91,91,Chelsea,€93M,€340K,2142,...,91.0,34.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0,€172.1M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,238813,J. Lundstram,19,England,47,65,Crewe Alexandra,€60K,€1K,1307,...,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,€143K
18203,243165,N. Christoffersson,19,Sweden,47,63,Trelleborgs FF,€60K,€1K,1098,...,42.0,22.0,15.0,19.0,10.0,9.0,9.0,5.0,12.0,€113K
18204,241638,B. Worman,16,England,47,67,Cambridge United,€60K,€1K,1189,...,41.0,32.0,13.0,11.0,6.0,5.0,10.0,6.0,13.0,€165K
18205,246268,D. Walker-Rice,17,England,47,66,Tranmere Rovers,€60K,€1K,1228,...,46.0,20.0,25.0,27.0,14.0,6.0,14.0,8.0,9.0,€143K


In [127]:
fifa.isnull().mean()

ID                0.000000
Name              0.000000
Age               0.000000
Nationality       0.000000
Overall           0.000000
                    ...   
GKHandling        0.002966
GKKicking         0.002966
GKPositioning     0.002966
GKReflexes        0.002966
Release Clause    0.088926
Length: 85, dtype: float64

In [128]:
#Verifico che le variabili che ho selezionato sono rimaste tutte quantitative.
fifa[['Age','Overall', 'Special', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing',
       'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing',
       'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions',
       'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Marking', 'StandingTackle', 'SlidingTackle']].dtypes

Age                           int64
Overall                       int64
Special                       int64
International Reputation    float64
Weak Foot                   float64
Skill Moves                 float64
Crossing                    float64
Finishing                   float64
HeadingAccuracy             float64
ShortPassing                float64
Volleys                     float64
Dribbling                   float64
Curve                       float64
FKAccuracy                  float64
LongPassing                 float64
BallControl                 float64
Acceleration                float64
SprintSpeed                 float64
Agility                     float64
Reactions                   float64
Balance                     float64
ShotPower                   float64
Jumping                     float64
Stamina                     float64
Strength                    float64
LongShots                   float64
Aggression                  float64
Interceptions               

In [129]:
#E vado a crearmi il mio subset
fifa=fifa[['Age','Overall', 'Special', 'Potential','International Reputation', 'Weak Foot', 'Skill Moves', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing',
       'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing',
       'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions',
       'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Marking', 'StandingTackle', 'SlidingTackle']]
fifa

,Age,Overall,Special,Potential,International Reputation,Weak Foot,Skill Moves,Crossing,Finishing,HeadingAccuracy,...,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle
0,31,94,2202,94,5.0,4.0,4.0,84.0,95.0,70.0,...,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0
1,33,94,2228,94,5.0,4.0,5.0,84.0,94.0,89.0,...,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0
2,26,92,2143,93,5.0,5.0,5.0,79.0,87.0,62.0,...,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0
4,27,91,2281,92,4.0,5.0,4.0,93.0,82.0,55.0,...,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0
5,27,91,2142,91,4.0,4.0,4.0,81.0,84.0,61.0,...,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,47,1307,65,1.0,2.0,2.0,34.0,38.0,40.0,...,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0
18203,19,47,1098,63,1.0,2.0,2.0,23.0,52.0,52.0,...,42.0,47.0,16.0,46.0,33.0,43.0,42.0,22.0,15.0,19.0
18204,16,47,1189,67,1.0,3.0,2.0,25.0,40.0,46.0,...,45.0,32.0,15.0,48.0,43.0,55.0,41.0,32.0,13.0,11.0
18205,17,47,1228,66,1.0,3.0,2.0,44.0,50.0,39.0,...,34.0,33.0,22.0,44.0,47.0,50.0,46.0,20.0,25.0,27.0


In [130]:
#Verifico i valori mancanti e noto che un gruppo di variabili presenta lo stesso numero di mancanti 
fifa.isnull().mean()

Age                         0.000000
Overall                     0.000000
Special                     0.000000
Potential                   0.000000
International Reputation    0.002966
Weak Foot                   0.002966
Skill Moves                 0.002966
Crossing                    0.002966
Finishing                   0.002966
HeadingAccuracy             0.002966
ShortPassing                0.002966
Volleys                     0.002966
Dribbling                   0.002966
Curve                       0.002966
FKAccuracy                  0.002966
LongPassing                 0.002966
BallControl                 0.002966
Acceleration                0.002966
SprintSpeed                 0.002966
Agility                     0.002966
Reactions                   0.002966
Balance                     0.002966
ShotPower                   0.002966
Jumping                     0.002966
Stamina                     0.002966
Strength                    0.002966
LongShots                   0.002966
A

In [131]:
#Verifichiamo che effettivamente sono gli stessi record ad avere dei valori mancanti
fifa[fifa.Balance.isnull()]

,Age,Overall,Special,Potential,International Reputation,Weak Foot,Skill Moves,Crossing,Finishing,HeadingAccuracy,...,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle
13236,33,62,1510,62,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13237,29,62,1532,62,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13238,35,62,1573,62,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13239,20,62,1610,72,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13240,24,62,1481,66,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13241,21,62,1692,72,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13242,23,62,1663,70,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13243,19,62,1328,78,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13244,30,62,1244,62,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13245,21,62,1549,69,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
#Vado ad eliminare le righe che presentano questi valori mancanti, sono di una quantità esigua (0,2% del dataset)
fifa=fifa.drop(index=[13236, 13237, 13238, 13239, 13240, 13241, 13242, 13243, 13244,
            13245, 13246, 13247, 13248, 13249, 13250, 13251, 13252, 13253,
            13254, 13255, 13256, 13257, 13258, 13259, 13260, 13261, 13262,
            13263, 13264, 13265, 13266, 13267, 13268, 13269, 13270, 13271,
            13272, 13273, 13274, 13275, 13276, 13277, 13278, 13279, 13280,
            13281, 13282, 13283])

fifa

,Age,Overall,Special,Potential,International Reputation,Weak Foot,Skill Moves,Crossing,Finishing,HeadingAccuracy,...,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle
0,31,94,2202,94,5.0,4.0,4.0,84.0,95.0,70.0,...,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0
1,33,94,2228,94,5.0,4.0,5.0,84.0,94.0,89.0,...,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0
2,26,92,2143,93,5.0,5.0,5.0,79.0,87.0,62.0,...,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0
4,27,91,2281,92,4.0,5.0,4.0,93.0,82.0,55.0,...,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0
5,27,91,2142,91,4.0,4.0,4.0,81.0,84.0,61.0,...,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,47,1307,65,1.0,2.0,2.0,34.0,38.0,40.0,...,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0
18203,19,47,1098,63,1.0,2.0,2.0,23.0,52.0,52.0,...,42.0,47.0,16.0,46.0,33.0,43.0,42.0,22.0,15.0,19.0
18204,16,47,1189,67,1.0,3.0,2.0,25.0,40.0,46.0,...,45.0,32.0,15.0,48.0,43.0,55.0,41.0,32.0,13.0,11.0
18205,17,47,1228,66,1.0,3.0,2.0,44.0,50.0,39.0,...,34.0,33.0,22.0,44.0,47.0,50.0,46.0,20.0,25.0,27.0


In [133]:
#Verifichiamo se ci sono ancora nulli 
fifa.isnull().mean()

Age                         0.0
Overall                     0.0
Special                     0.0
Potential                   0.0
International Reputation    0.0
Weak Foot                   0.0
Skill Moves                 0.0
Crossing                    0.0
Finishing                   0.0
HeadingAccuracy             0.0
ShortPassing                0.0
Volleys                     0.0
Dribbling                   0.0
Curve                       0.0
FKAccuracy                  0.0
LongPassing                 0.0
BallControl                 0.0
Acceleration                0.0
SprintSpeed                 0.0
Agility                     0.0
Reactions                   0.0
Balance                     0.0
ShotPower                   0.0
Jumping                     0.0
Stamina                     0.0
Strength                    0.0
LongShots                   0.0
Aggression                  0.0
Interceptions               0.0
Positioning                 0.0
Vision                      0.0
Penaltie

In [134]:
#Creo la mia matrice di variabili predittive e la trasformo in array
X=fifa[['Age','Overall', 'Special', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing',
       'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing',
       'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions',
       'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Marking', 'StandingTackle', 'SlidingTackle']].values
X.shape

(16134, 35)

In [135]:
#individuo la mia variabile target trasformandola in array
y=fifa.Potential.values
y.shape

(16134,)

In [136]:
#Effettuo lo split tra training e test set che saranno ripartite in  80% e 20% del dataset. Indico un random state a caso
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1,test_size=0.2)

In [137]:
# Verifichiamo le dimensioni dopo lo split (12907+ 3227=16134)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
y_test.shape

(12907, 35)
(3227, 35)
(12907,)


(3227,)

In [138]:
#Import il mio modello di regressione lineare e adatto ai datidi training
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [139]:
#Vado a predire i nuovi valori sui dati di test
y_pred=model.predict(X_test)
y_pred

array([84.226259  , 76.47751623, 76.52527188, ..., 75.04392407,
       73.9021651 , 74.82057607])

In [140]:
#Vediamo i parametri del modello
print(model.coef_)#
model.intercept_

[-9.73757892e-01  8.64739769e-01 -1.53233732e-02  8.23237546e-01
  5.75823818e-02  8.03347492e-04  5.41619344e-03  1.53008063e-02
  2.08581255e-02  2.42880198e-02  1.12701633e-02  1.93756339e-02
  1.98754028e-02  1.19604079e-02 -5.10379544e-03  2.36086525e-02
  6.33634036e-04 -5.07255031e-03  8.41645355e-03  8.03673160e-03
  2.80859693e-02  1.33504093e-02  1.09194890e-02 -2.57494458e-02
 -5.68626847e-03  1.23134739e-02  1.67462348e-02  1.15218758e-02
  8.26897602e-03  2.74423091e-02  3.78661126e-02  2.98316807e-02
  2.74481362e-02  7.32747996e-03  3.37761607e-02]


41.12918440770038

Ogni parametro rappresenta il parametro di una specifica variabile β del modello ($ β_0$ rappresenta l'intercetta) nella formula:

$y=β_0 +β_1X_1+....+ β_pX_p + $


In [141]:
#Vediamo i parametri associati
a=model.coef_
modello=pd.DataFrame(a)
modello.columns=["Coef"]
modello

b=['Age','Overall', 'Special', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing',
       'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing',
       'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions',
       'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Marking', 'StandingTackle', 'SlidingTackle']
b=pd.DataFrame(b)
b.columns=["Variabile"]
Schema=coef.join(b)
Schema.loc[35,"Coef"]=[model.intercept_]
Schema.loc[35,"Variabile"]="Intercetta"
Schema

,Coef,Variabile
0,-0.973758,Age
1,0.864740,Overall
2,-0.015323,Special
3,0.823238,International Reputation
4,0.057582,Weak Foot
5,0.000803,Skill Moves
6,0.005416,Crossing
7,0.015301,Finishing
8,0.020858,HeadingAccuracy
9,0.024288,ShortPassing


In [142]:
#Vediamo Le variabili che influenzano maggiormente nel modello
Schema.sort_values("Coef", ascending = False).head(5)

,Coef,Variabile
35,41.129184,Intercetta
1,0.864740,Overall
3,0.823238,International Reputation
4,0.057582,Weak Foot
30,0.037866,Penalties


In [143]:
Schema.sort_values("Coef").head(5)

,Coef,Variabile
0,-0.973758,Age
23,-0.025749,Stamina
2,-0.015323,Special
24,-0.005686,Strength
14,-0.005104,LongPassing


In [144]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.8419684641208851

In [145]:
atteso= pd.DataFrame(y_pred)
a= pd.DataFrame(y_test)
atteso.columns=["Predetto"]
atteso=atteso.join(a)
atteso.columns=["Valore Predetto","Valore Effettivo"]

In [146]:
atteso

,Valore Predetto,Valore Effettivo
0,84.226259,84
1,76.477516,77
2,76.525272,79
3,67.794962,68
4,70.824065,70
...,...,...
3222,64.818360,64
3223,67.281154,66
3224,75.043924,81
3225,73.902165,74


# Permutation importance

I punteggi di importanza delle caratteristiche **(Permutation importance)** giocano un ruolo importante in un progetto di modellazione predittiva, incluso il fornire informazioni sui dati, informazioni sul modello e la base per la riduzione della dimensionalità e la selezione delle caratteristiche che possono migliorare l'efficienza e l'efficacia di un modello predittivo sul problema.
L'importanza della caratteristica di permutazione è una tecnica di ispezione del modello che può essere utilizzata per qualsiasi stimatore adattato quando i dati sono tabulari. LE' è definita come la diminuzione del punteggio di un modello quando un singolo valore della caratteristica viene mescolato casualmente.  Questa tecnica trae vantaggio dall'essere indipendente dal modello e può essere calcolata molte volte con diverse permutazioni della  stessa caratteristica.

Le caratteristiche che sono di scarsa importanza per un modello potrebbero essere molto importanti per un buon modello. E' sempre opportuno valutare precedentemente la bontà di un modello con un insieme di variabili magari effettuando una cross validation prima di valutare l'importanza delle variabili. La permutation importance non riflette il valore predittivo intrinseco di una variabile di un modello ma quanto quella variabile è importante per il modello.
Da fare attenzione al problema della *collinearità*. Se due variabili sono correlate avrò un punteggio più basso per entrambe

In [147]:
import eli5

In [148]:
from eli5.sklearn import PermutationImportance

In [149]:
b=['Age','Overall', 'Special', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing',
       'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing',
       'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions',
       'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Marking', 'StandingTackle', 'SlidingTackle']
perm = PermutationImportance(model, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = b)

Weight,Feature
1.8636 ± 0.0237,Overall
1.0742 ± 0.0262,Age
0.4961 ± 0.0213,Special
0.0192 ± 0.0020,SlidingTackle
0.0134 ± 0.0021,Penalties
0.0092 ± 0.0018,Marking
0.0071 ± 0.0019,Vision
0.0065 ± 0.0011,International Reputation
0.0058 ± 0.0008,Balance
0.0056 ± 0.0011,Curve
